In [1]:
import numpy as np

In [2]:
# other test variables, which are self parameters
# classes schould be given as int from 0 to k
n_total_ftr = 3
target_values = [1,2]
mu = np.ones((n_total_ftr, len(target_values))) * 0
sigma = np.ones((n_total_ftr, len(target_values))) * 1
penalty_s = 0.01
penalty_r = 0.01
epochs = 1
lr_mu = 0.01
lr_sigma = 0.01
no_mc_samples = 2 #10000 # monte carlo samples

In [54]:
def __softmax(x,y): # needs self in model
    # sort observations
    observed_classes = np.unique(y)

    for obs_class in observed_classes:

        observations_index = np.where(y == obs_class)[0]
        x_obs = x[observations_index]
        n_observations = len(x_obs)
    
        for epoch in range(epochs): #changed to self.epoch in model
                
                # Iterative update of mu and sigma
                try:
                    # o number of obs, l number of samples, j features, c classes
                    # create 3d array with all r for current observation for multiple observation calculation we would need 4d array
                    # r^cl_j = r[l, j, c] lxjxc
                    r = np.random.randn(n_obs, no_mc_samples, n_total_ftr, len(target_values))
                    #r = np.array([[[-0.558, 1.555], [0.325, -0.726], [0.347,-0.159]],
                    #              [[-0.955, 0.283], [0.115,-1.637], [-0.516,0.161]]])  
                    print(r)
                    # we only change the psi for the actuall given class still need all classes of course
                    # r = np.random.randn(monte_carlo, n_total_ftr)

                    print(r.shape)
                    # calculate thetas for all samples and classes theta^cl_jt = theta[l,j,c]
                    # oxlxjxc
                    theta = r * sigma + mu
                    print(theta.shape)
                    print(theta)
                    #calculate all the etas
                    eta = np.einsum("oljc,oj->oljc", theta, x) # multiply all ftr_cols with given ftr_vector x
                    print('theta * x:')
                    print(eta)
                    eta = np.einsum("oljc->olc", eta) #sum up all theta^cl_j * x_tj so we got l samples for all c classes
                    print("eta:")
                    print(eta)
                    eta = np.exp(eta) # we only need them exp
                    print("eta_exp:")
                    print(eta)
                    print(eta.shape)
                    eta_sum = np.einsum("olc->ol", eta) #sum up etas for the l samples
                    print("eta_sum:")
                    print(eta_sum)
                    print(eta_sum.shape)
                    #calculate softmax only for observed class
                    #observation_etas = eta[:,y] with more observations we need transposition
                    obs_eta = eta[:,:,obs_class]
                    print("Obs_eta shape: {}".format(obs_eta.shape))
                    softmax_lh = obs_eta / eta_sum # 
                    print(softmax_lh)
                    print(softmax_lh.shape) #should be oxl
                    #marginal = np.einsum("lo->o", softmax_lh) / monte_carlo # 1xy
                    marginal = np.einsum("ol->o", softmax_lh) / no_mc_samples
                    print(marginal.shape) #should be o
                    print(marginal)
                    # calculate derivatives nabla_mu, nabla_sigma must be handled better

                    #first calculate softmax dtheta
                    # x_eta means observations x times the beloning etas
                    x_eta = np.einsum("oj,ol->olj", x_obs, obs_eta)
                    print(x_eta)

                    softmax_derivative = np.einsum("olj,ol->olj", x_eta, (eta_sum - obs_eta))
                    softmax_derivative = np.einsum("olj->jol", softmax_derivative) /  eta_sum**2
                    softmax_derivative = np.einsum("jol->olj", softmax_derivative)
                    print(softmax_derivative.shape)
                    print(softmax_derivative)
                    nabla_mu = np.einsum("olj->oj", softmax_derivative) / no_mc_samples
                    print(nabla_mu.shape) #oj
                    print(nabla_mu)
                    r_jc = r[:,:,:,obs_class]
                    print(r_jc.shape)
                    nabla_sigma = np.einsum("olj->oj", softmax_derivative * r_jc) / no_mc_samples
                    print(nabla_sigma.shape) #oj
                    # Update parameters
                    mu[:,obs_class] += lr_mu * np.einsum("jo->j", (nabla_mu.T / marginal))
                    sigma[:,obs_class] += lr_sigma * np.einsum("jo->j",(nabla_sigma.T / marginal))
                    

                except TypeError as e:
                        raise TypeError('All features must be a numeric data type.') from e

In [6]:
for i in np.unique(np.array((1,1))):
    print(i)

1


In [10]:
np.where(np.array((1,2,3,1,2,4,3,2)) == 2)[0]

array([1, 4, 7])

In [17]:
theta = np.ones((2,2,2,2)) * np.array(([1,2],[3,4]))
x = np.array(([1,2], [3,4]))
print("x: {} theta: {}".format(x.shape, theta.shape))
np.einsum("oljc,oj->oljc", theta, x)

x: (2, 2) theta: (2, 2, 2, 2)


array([[[[ 1.,  2.],
         [ 6.,  8.]],

        [[ 1.,  2.],
         [ 6.,  8.]]],


       [[[ 3.,  6.],
         [12., 16.]],

        [[ 3.,  6.],
         [12., 16.]]]])

In [18]:
np.ones((2,2)) / np.array(([1,2],[3,4]))

array([[1.        , 0.5       ],
       [0.33333333, 0.25      ]])

In [22]:
# test for obs_eta eta[:,:,y]
test = np.array([[[1,2,3],[1,2,3]],[[1,2,3],[1,2,3]]])
test
test[:,:,0]

array([[[1, 2, 3],
        [1, 2, 3]],

       [[1, 2, 3],
        [1, 2, 3]]])

In [45]:
#test calculate derivative
test =np.array((np.array((np.array((1,2,3,4)), np.array((4,5,6,7)))), np.array((np.array((1,2,3,4)), np.array((4,5,6,7)))), np.array((np.array((1,2,3,4)), np.array((4,5,6,7))))))
print(test)
test = np.einsum("ojl->jol",test)
print(test)
der = test / np.array(([1,1,1,1],[2,2,2,2],[3,3,3,3]))
print(der)
np.einsum("jol->ojl", der)

[[[1 2 3 4]
  [4 5 6 7]]

 [[1 2 3 4]
  [4 5 6 7]]

 [[1 2 3 4]
  [4 5 6 7]]]
[[[1 2 3 4]
  [1 2 3 4]
  [1 2 3 4]]

 [[4 5 6 7]
  [4 5 6 7]
  [4 5 6 7]]]
[[[1.         2.         3.         4.        ]
  [0.5        1.         1.5        2.        ]
  [0.33333333 0.66666667 1.         1.33333333]]

 [[4.         5.         6.         7.        ]
  [2.         2.5        3.         3.5       ]
  [1.33333333 1.66666667 2.         2.33333333]]]


array([[[1.        , 2.        , 3.        , 4.        ],
        [4.        , 5.        , 6.        , 7.        ]],

       [[0.5       , 1.        , 1.5       , 2.        ],
        [2.        , 2.5       , 3.        , 3.5       ]],

       [[0.33333333, 0.66666667, 1.        , 1.33333333],
        [1.33333333, 1.66666667, 2.        , 2.33333333]]])

In [24]:
 a = np.array([[1,2],[3,4]])
 np.dot(a,a)

array([[ 7, 10],
       [15, 22]])

In [48]:
np.einsum("olj->oj",test)

array([[ 3,  6,  9, 12],
       [12, 15, 18, 21]])

In [50]:
test.shape

(2, 3, 4)

In [53]:
np.array(([1,2,3],[1,2,3])).T / np.array((1,2))

array([[1. , 0.5],
       [2. , 1. ],
       [3. , 1.5]])